In [1]:
!pip3 install requests
import requests
import json
import pandas as pd
from pandas import json_normalize
import datetime

In [2]:
#url for Police Department Incident Reports: 2018 to Present (dataset #1)
url = "https://data.sfgov.org/resource/wg3w-h783.json?$limit=1000000"
#url for Police Department Incident Reports: Historical 2003 to May 2018 (dataset #2)
url_2 = "https://data.sfgov.org/resource/tmnf-yvry.json?$limit=2000000"

In [3]:
#returns a JSON object of the result (url)
response = requests.get(url).json()

In [4]:
#returns a JSON object of the result (url_2)
response_2 = requests.get(url_2).json()

In [5]:
#Normalize semi-structured JSON data into a flat table. (df)
res = json_normalize(response)
df = pd.DataFrame(res)

In [6]:
##Normalize semi-structured JSON data into a flat table. (df_2)
res_2 = json_normalize(response_2)
df_2 = pd.DataFrame(res_2)

In [7]:
#extract the months from incident date before dropping it (dataset #1)
df['incident_month'] = pd.DatetimeIndex(df['incident_date']).month

#dropping columns
df = df.drop(['incident_datetime', 'row_id','report_type_description',
'report_datetime','incident_time','incident_date','incident_day_of_week',':@computed_region_nqbw_i6c3',
 ':@computed_region_h4ep_8xdi','report_type_code',
 ':@computed_region_n4xg_c4py',
 ':@computed_region_jg9y_a9du',
 ':@computed_region_jwn9_ihcz',
 ':@computed_region_26cr_cadq','analysis_neighborhood',
 ':@computed_region_qgnn_b9vv', 'police_district','incident_subcategory',
 'point.type', 'supervisor_district', 'resolution','filed_online','incident_number', 'incident_id', 'cad_number', 'cnn', 'intersection','point.coordinates',], axis = 1)

In [8]:
#dropping null values
df = df.dropna(how="any")

#dropping 2023 data and reset the index
df = df.drop(df.index[df['incident_year'] == "2023"])
df = df.reset_index(drop = True)

In [9]:
#extract the months and years from incident date before dropping it (dataset #2)

df_2['incident_month'] = pd.DatetimeIndex(df_2['date']).month
df_2['incident_year'] = pd.DatetimeIndex(df_2['date']).year

In [10]:
#dropping columns (dataset #2)
df_2= df_2.drop(['time','resolution', 'location.coordinates', 'location.type', ':@computed_region_26cr_cadq',
 ':@computed_region_rxqg_mtj9',
 ':@computed_region_bh8s_q3mv','dayofweek','pdid','pddistrict',
 'incidntnum',':@computed_region_6qbp_sg9q',
 ':@computed_region_qgnn_b9vv','date',
 'address',
 ':@computed_region_ajp5_b2md',
 ':@computed_region_yftq_j783',
 ':@computed_region_p5aj_wyqh',
 ':@computed_region_fyvs_ahh9',
 ':@computed_region_6pnf_4xz7',
 ':@computed_region_jwn9_ihcz',
 ':@computed_region_9dfj_4gjx',
 ':@computed_region_4isq_27mq',
 ':@computed_region_pigm_ib2e',
 ':@computed_region_9jxd_iqea',
 ':@computed_region_6ezc_tdp2',
 ':@computed_region_h4ep_8xdi',
 ':@computed_region_n4xg_c4py',
 ':@computed_region_fcz8_est8',
 ':@computed_region_nqbw_i6c3',
 ':@computed_region_2dwj_jsy4'], axis = 1)

In [11]:
#rename columns (dataset #2)
df_2.rename(columns = {'category':'incident_category', 'descript':'incident_description', 'y':'latitude', 'x':'longitude'}, inplace = True)

In [12]:
#dropping null values (dataset #2)
df_2 = df_2.dropna(how="any")
#changing incident_year type to mearge the two Dataframes (dataset #2)
df_2.astype({'incident_year': 'object'},{}).dtypes

incident_code           object
incident_category       object
incident_description    object
longitude               object
latitude                object
incident_month           int64
incident_year           object
dtype: object

In [13]:
#creating a combined Dataframe 
crime_df = pd.concat([df, df_2], ignore_index=True, sort=False)
crime_df.astype({'incident_category': 'str'},{'incident_description' : 'str'}).dtypes

incident_year           object
incident_code           object
incident_category       object
incident_description    object
latitude                object
longitude               object
incident_month           int64
dtype: object

In [14]:
# Display the new Dataframe created
crime_df

,incident_year,incident_code,incident_category,incident_description,latitude,longitude,incident_month
0,2021,75000,Missing Person,Found Person,37.72623624315635,-122.43362359230944,5
1,2021,09320,Fraud,"Access Card, incl. Credit, Phone, ATM, Fraudul...",37.80549664761133,-122.42200682265661,5
2,2021,65010,Traffic Violation Arrest,Traffic Violation Arrest,37.77279045274103,-122.44749724585684,5
3,2021,27195,Other Miscellaneous,Trespassing,37.78983697125977,-122.43402709034117,5
4,2021,16710,Drug Offense,"Narcotics Paraphernalia, Possession of",37.73985319897475,-122.39126842523832,5
...,...,...,...,...,...,...,...
2643556,2012,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,37.7849531241612,-122.427991151911,8
2643557,2016,04012,ASSAULT,AGGRAVATED ASSAULT WITH A KNIFE,37.7531451876949,-122.43094215775518,4
2643558,2006,06304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,37.7807425792871,-122.455063675296,6
2643559,2005,05011,BURGLARY,"BURGLARY OF APARTMENT HOUSE, FORCIBLE ENTRY",37.7929373231049,-122.430448772333,8


In [15]:
# Changing letters to lower case in both incident_category & incident_description columns
crime_df["incident_category"] = crime_df["incident_category"].str.lower()
crime_df["incident_description"] = crime_df["incident_description"].str.lower()

#know all type of unique values in column
crime_df["incident_category"].unique()

array(['missing person', 'fraud', 'traffic violation arrest',
       'other miscellaneous', 'drug offense', 'motor vehicle theft',
       'other offenses', 'drug violation', 'burglary', 'non-criminal',
       'robbery', 'warrant', 'larceny theft', 'recovered vehicle',
       'malicious mischief', 'arson', 'stolen property', 'assault',
       'weapons offense', 'disorderly conduct', 'suspicious occ',
       'courtesy report', 'case closure', 'fire report',
       'weapons carrying etc', 'lost property',
       'miscellaneous investigation', 'vandalism',
       'forgery and counterfeiting',
       'offences against the family and children', 'sex offense', 'other',
       'traffic collision', 'vehicle impounded', 'vehicle misplaced',
       'embezzlement', 'human trafficking (a), commercial sex acts',
       'civil sidewalks', 'homicide', 'prostitution', 'suicide', 'rape',
       'liquor laws', 'weapons offence', 'suspicious',
       'motor vehicle theft?', 'gambling',
       'human traff

In [16]:
# replace values.
crime_df['incident_category'] = crime_df['incident_category'].replace 


(['drug violation', 'drug/narcotic','forgery/counterfeiting','larceny/theft','human trafficking, commercial sex acts','motor vehicle theft?','other miscellaneous','other offenses','sex offenses, forcible','sex offenses, non forcible','suspicious occ','warrants','weapons offence','weapons carrying etc'],
 
 
 
 ['drug offense','drug offense','forgery and counterfeiting','larceny theft','human trafficking (a), commercial sex acts','motor vehicle theft','other','other','sex offense','sex offense','suspicious','warrant','weapons offense','weapon laws'])


In [17]:
# Display dataframe after replacing values. 
crime_df

,incident_year,incident_code,incident_category,incident_description,latitude,longitude,incident_month
0,2021,75000,missing person,found person,37.72623624315635,-122.43362359230944,5
1,2021,09320,fraud,"access card, incl. credit, phone, atm, fraudul...",37.80549664761133,-122.42200682265661,5
2,2021,65010,traffic violation arrest,traffic violation arrest,37.77279045274103,-122.44749724585684,5
3,2021,27195,other,trespassing,37.78983697125977,-122.43402709034117,5
4,2021,16710,drug offense,"narcotics paraphernalia, possession of",37.73985319897475,-122.39126842523832,5
...,...,...,...,...,...,...,...
2643556,2012,06244,larceny theft,grand theft from locked auto,37.7849531241612,-122.427991151911,8
2643557,2016,04012,assault,aggravated assault with a knife,37.7531451876949,-122.43094215775518,4
2643558,2006,06304,larceny theft,grand theft from a building,37.7807425792871,-122.455063675296,6
2643559,2005,05011,burglary,"burglary of apartment house, forcible entry",37.7929373231049,-122.430448772333,8
